In [514]:
import bempp.api
import numpy as np
bempp.api.PLOT_BACKEND = "paraview"

In [515]:
# parametros 
num_base = 65 #numero de iteraciones 30

In [516]:
grid = bempp.api.shapes.sphere(h=0.1)
grid.plot()

In [517]:
print("numero de elementos:", grid.number_of_elements)
print("numero de bordes:", grid.number_of_edges)
print("numero de vertices:", grid.number_of_vertices)

numero de elementos: 3216
numero de bordes: 4824
numero de vertices: 1610


In [518]:
vertices = grid.vertices

# Imprimir los vértices
for i in range(vertices.shape[1]):
    print(f"Vértice {i}: {vertices[:, i]}")
scale_factor = np.random.uniform(0.98, 1)
vertices[0, :] *= scale_factor # escalar en la dirección x
vertices[1, :] *= scale_factor # escalar en la dirección y

Vértice 0: [1. 0. 0.]
Vértice 1: [0. 1. 0.]
Vértice 2: [-1.  0.  0.]
Vértice 3: [ 0. -1.  0.]
Vértice 4: [ 0.  0. -1.]
Vértice 5: [0. 0. 1.]
Vértice 6: [0.99518473 0.09801714 0.        ]
Vértice 7: [0.98078528 0.19509032 0.        ]
Vértice 8: [0.95694034 0.29028468 0.        ]
Vértice 9: [0.92387953 0.38268343 0.        ]
Vértice 10: [0.88192126 0.47139674 0.        ]
Vértice 11: [0.83146961 0.55557023 0.        ]
Vértice 12: [0.77301045 0.63439329 0.        ]
Vértice 13: [0.70710678 0.70710678 0.        ]
Vértice 14: [0.63439328 0.77301045 0.        ]
Vértice 15: [0.55557023 0.83146961 0.        ]
Vértice 16: [0.47139674 0.88192127 0.        ]
Vértice 17: [0.38268343 0.92387953 0.        ]
Vértice 18: [0.29028468 0.95694034 0.        ]
Vértice 19: [0.19509032 0.98078528 0.        ]
Vértice 20: [0.09801714 0.99518473 0.        ]
Vértice 21: [-0.09801714  0.99518473  0.        ]
Vértice 22: [-0.19509032  0.98078528  0.        ]
Vértice 23: [-0.29028468  0.95694034  0.        ]
Vértice 

In [519]:
num_vertices = grid.vertices.shape[1]
num_elementos = grid.number_of_elements
random_vector = np.round(np.random.uniform(0.96, 1, num_vertices), 4)
print(random_vector)

[0.987  0.9832 0.9865 ... 0.9762 0.9914 0.9633]


In [520]:
# for i in range(num_vertices):
#     vertices[:, i] += vertices[:, i] * random_vector[i]

# new_grid = bempp.api.Grid(vertices, grid.elements)
# new_grid.plot()

In [521]:
v_perturbacion = np.empty((0, num_vertices))
A = np.empty((0, num_vertices))

for sup in range(num_base):
    num_vertices = grid.vertices.shape[1]
    random_vector = np.round(np.random.uniform(0, 0.02, num_vertices), 4)
    eps = np.round(np.random.uniform(0, 0.5), 4)
    print(random_vector)
    grid = bempp.api.shapes.sphere(h=0.1)
    vertices = grid.vertices
    for i in range(num_vertices):
        vertices[:, i] += vertices[:, i] * (random_vector[i]) #perturbar con el vector de coordenadas esfericas cos de t a cos de 2t  #* random_vector[i]   #* (random_vector[i] - 0.001) * (eps)
    scale_factor = np.random.uniform(1, 1.5)
    vertices[0, :] *= scale_factor # escalar en la dirección x
    vertices[1, :] *= scale_factor # escalar en la dirección y
    
    v_perturbacion = np.vstack([v_perturbacion, random_vector])
    new_grid = bempp.api.Grid(vertices, grid.elements)
    # new_grid.plot()
    dp0_space = bempp.api.function_space(new_grid, "DP", 0)
    p1_space = bempp.api.function_space(new_grid, "P", 1)
    identity = bempp.api.operators.boundary.sparse.identity(
        p1_space, p1_space, p1_space)     # p1_space, p1_space, dp0_space
    dlp = bempp.api.operators.boundary.laplace.double_layer(
        p1_space, p1_space, p1_space)  # p1_space, p1_space, dp0_space
    slp = bempp.api.operators.boundary.laplace.single_layer(
        p1_space, p1_space, p1_space) #dp0_space, p1_space, dp0_space

    @bempp.api.real_callable
    def dirichlet_data(x, n, domain_index, result):
        result[0] = np.cos(x[0] + x[1]) #np.cos(x[0] + x[1])   #= 1./(4 * np.pi * ((x[0] - .9)**2 + x[1]**2 + x[2]**2)**(0.5))
        
    dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)

    rhs = (.5 * identity + dlp) * dirichlet_fun
    neumann_fun, info = bempp.api.linalg.cg(slp, rhs, tol=1E-3)
    print(neumann_fun)
    neumann_vector = neumann_fun.coefficients
    A = np.vstack([A, neumann_vector])
    print(neumann_vector)

[0.0177 0.0027 0.0092 ... 0.0036 0.0056 0.0115]


c:\Users\Benja\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:394: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


[-0.53682105 -0.47895527 -0.40955827 ...  1.00413544  0.23598533
  0.61535997]
[0.0023 0.0041 0.0148 ... 0.0159 0.0069 0.0144]



c:\Users\Benja\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:394: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


[-0.62965387 -0.61803472 -0.81873847 ...  0.74904006  0.06241448
  0.71942509]
[0.0184 0.0017 0.0178 ... 0.0017 0.0117 0.0046]



...

[-0.40689647 -0.42211694 -0.40776691 ...  0.70074603  0.35309213
  0.66709269]
[0.0075 0.015  0.0067 ... 0.0053 0.0128 0.0179]



c:\Users\Benja\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:394: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


[-0.44495243 -0.48024804 -0.38801626 ...  0.8230534   0.19087748
  0.40229291]
[0.0038 0.0103 0.018  ... 0.0142 0.0139 0.0063]



c:\Users\Benja\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:394: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


[-0.59089591 -0.78225057 -0.75999266 ...  0.70397291 -0.11735682
  0.90761191]


In [522]:
A = A.T
v_perturbacion = v_perturbacion.T
v_perturbacion.shape
A.shape
A

array([[-0.53682105, -0.62965387, -0.39640108, ..., -0.40689647,
        -0.44495243, -0.59089591],
       [-0.47895527, -0.61803472, -0.23457354, ..., -0.42211694,
        -0.48024804, -0.78225057],
       [-0.40955827, -0.81873847, -0.52239682, ..., -0.40776691,
        -0.38801626, -0.75999266],
       ...,
       [ 1.00413544,  0.74904006,  0.78301127, ...,  0.70074603,
         0.8230534 ,  0.70397291],
       [ 0.23598533,  0.06241448,  0.13407159, ...,  0.35309213,
         0.19087748, -0.11735682],
       [ 0.61535997,  0.71942509,  0.56314209, ...,  0.66709269,
         0.40229291,  0.90761191]])

In [523]:
U, S, Vt = np.linalg.svd(A)

print("U:", U)
print("S:", S)
print("Vt:", Vt)

U: [[-2.17041864e-02 -4.28654046e-02 -1.32453120e-02 ...  3.58624529e-02
   1.24419666e-03  1.60676240e-02]
 [-2.27361994e-02 -4.73609782e-02 -4.98570634e-03 ... -5.76546127e-02
  -3.53227265e-02  8.82301221e-03]
 [-2.22428739e-02 -6.37593113e-02 -9.80317522e-03 ...  7.53715752e-03
   1.53903144e-03  8.25948550e-03]
 ...
 [ 3.23439707e-02  1.60904050e-02  2.79074253e-02 ...  9.45645072e-01
   1.61811245e-03 -8.00982236e-03]
 [ 8.26958276e-03 -5.38389108e-02  2.66087598e-05 ... -5.52224193e-04
   9.36637928e-01 -2.02123585e-03]
 [ 2.77429423e-02  3.22979343e-02  6.79649185e-03 ... -9.59178662e-03
   3.03061489e-03  9.45311870e-01]]
S: [191.17371039  11.42258499   5.27386818   5.14065284   5.10946816
   5.04899913   4.97058288   4.94204735   4.88437823   4.88013266
   4.82848011   4.80719336   4.79313909   4.7828928    4.74140903
   4.67418317   4.66158829   4.64354775   4.59169793   4.58516499
   4.5426953    4.53366215   4.50888723   4.4933473    4.4605957
   4.44990496   4.41484018   

In [524]:
# (1-((S[0])**2)/((np.sum(S))**2))**0.5 #tolerancia

In [525]:
U_1 = U[:, :30]
perturbacion_L = np.round(np.random.uniform(0, 0.02, num_vertices), 4)
eps_L = np.round(np.random.uniform(0, 3), 4) #0.5
grid = bempp.api.shapes.sphere(h=0.1)
vertices = grid.vertices
for i in range(num_vertices):
        vertices[:, i] += vertices[:, i] * (perturbacion_L[i])  #(perturbacion_L[i] - 0.001) * (eps_L)
scale_factor = np.random.uniform(1, 1.5) #1.5
vertices[0, :] *= scale_factor # escalar en la dirección x
vertices[1, :] *= scale_factor # escalar en la dirección y
grid_L = bempp.api.Grid(vertices, grid.elements)
# grid_L.plot()
dp0_space_L = bempp.api.function_space(grid_L, "DP", 0)
p1_space_L = bempp.api.function_space(grid_L, "P", 1)
V_L = bempp.api.operators.boundary.laplace.single_layer(
    p1_space_L, p1_space_L, p1_space_L)  #dp0_space_L, p1_space_L, dp0_space_L
V_L_matrix = V_L.weak_form().A

identity_L = bempp.api.operators.boundary.sparse.identity(
    p1_space_L, p1_space_L, p1_space_L)  #p1_space_L, p1_space_L, dp0_space_L
dlp_L = bempp.api.operators.boundary.laplace.double_layer(
    p1_space_L, p1_space_L, p1_space_L) #p1_space_L, p1_space_L, dp0_space_L

@bempp.api.real_callable
def dirichlet_data(x, n, domain_index, result):
    result[0] = np.cos(x[0]+ x[1])   #1./(4 * np.pi * ((x[0] - .9)**2 + x[1]**2 + x[2]**2)**(0.5))

dirichlet_fun_L = bempp.api.GridFunction(p1_space_L, fun=dirichlet_data)
rhs_L = (.5 * identity_L + dlp_L) * dirichlet_fun_L

neumann_fun_L, info_L = bempp.api.linalg.cg(V_L, rhs_L, tol=1E-3)
neumann_vector_L = neumann_fun.coefficients

c:\Users\Benja\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:394: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


In [526]:
rhs_L_vector = rhs_L.projections(p1_space_L)
identity_L_matrix = identity_L.weak_form().A


In [527]:
# from scipy.sparse import csc_matrix
# from scipy.sparse.linalg import spsolve

# I = csc_matrix(np.eye(*identity_L_matrix.shape))
# identity_L_matrix_inv = spsolve(identity_L_matrix, I)

In [528]:

a = U_1.T  @ V_L_matrix @ U_1 #@ identity_L_matrix_inv
b = U_1.T  @ rhs_L_vector

# Resolviendo para x
x = np.linalg.solve(a, b)

In [529]:
(np.linalg.norm((U_1 @ x) - neumann_vector_L)) / (np.linalg.norm(neumann_vector_L)) #error 0.1458403 con 30, con 40 0.0673

0.1581764271279679

In [530]:

grid_L.plot()